# Quantum Variational Simulator

## Summary

The usual method to simulate quantum evolution in quantum systems is using the Trotter expansion. However, this usually requires circuits with many quantum operations and a large depth, which is not desirable in the current noisy era of quantum computation. An alternative is to use Variational Quantum Simulation, since the depth of the circuit is fixed during all the computation and we can choose variational forms that are efficient for different hardwares. 

In this project, we implement a Python package that, given a Hamiltonian, a variational form and a state at t=0, gives us the state of the system for a future state at time t = T using a variational method. In the tutorial, we present the theory of variational quantum simulation and continue with the details of our implementation using Qiskit. Then, we show simulations for the evolution of quantum Isings models of 2 and 3 qubits and compare with experimental realizations using IBM's quantum processors and Qiskit Runtime. The results show an excellent agreement between theory and experiment.

## 1. Introduction

The problem of simulating quantum many-body systems is well-known to be extremely challenging for classical computers. This is due to the exponential dimension of the associated Hilbert space. For these systems, it is impossible to diagonalize the Hamiltonian to find the ground state energy, or to solve the Schrödinger equation to obtain its time evolution. Variational Classical Simulation (VCE) $[$[1](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)), [2](https://doi.org/10.1016/0003-4916(88)90280-1), [3](https://doi.org/10.1103/PhysRevLett.107.070601)$]$ is an alternative to these direct methods because effectively reduces the dimension of systems using efficient classical representations. It can be used to find the ground state energy and to simulate time evolution of quantum systems. However, for highly-entangled systems VQE is usually infeasible, because there are not good classical representations of these states. 

The efficient alternative is using quantum systems to simulate quantum systems. The Trotterization technique is the usual method to perform quantum evolution, and approximates the time evolution under a Hamiltonian $H = \sum_j H_j$ from $0$ to $T$ by

$$
e^{-i H T} \approx   \left( \prod_j e^{-i H_j \Delta t}\right)^N,
$$

where $e^{-i H_j \Delta t}$ is the unitary generated by $H_j$ for a short time $\Delta t = T/N$. Variational Quantum Algorithms (VQA) are the quantum version of VCE and they can be used to find the ground state energy of relevant Hamiltonians using efficient variational forms and classical optimization. These algorithms can also be used as an alternative for Trotterization, that usually requires circuits with many quantum operations and a large depth. This is not desirable in the current era of quantum computation, because quantum processors are still noisy and the depth of the circuits is restricted by the decoherence time of the system. For VQA, the depth of the circuit is fixed during all the computation and we can choose variational forms that are efficient for different hardwares.

In this project, we present a Python package that, given a Hamiltonian, a variational form and a state at $t=0$, gives us the state of the system for a future state at time $t = T$ using the variational method presented in the paper [Efficient Variational Quantum Simulator Incorporating Active Error Minimization](https://doi.org/10.1103/PhysRevX.7.021050). We start this tutorial with the theory of variational quantum simulation and continue with the details of our implementation using Qiskit. Then, we show simulations for the evolution of quantum Isings models of 2 and 3 qubits and compare with experimental realizations using IBM's quantum processors and Qiskit Runtime. Finally, we discuss our results and propose future improvements of the package.

## 2. The time-dependent variational principle

This principle corresponding to the Schrödinger equation reads $\delta \int_{t_{i}}^{t_{f}} dt L=0$, where the Lagrangian is $L=\langle\Psi(t)|\left(i \frac{\partial}{\partial t}-H\right)| \Psi(t)\rangle$. Assuming that the state $| \Psi(t)\rangle$ is determined by a set of real parameters $\left\{\lambda_{k}(t)\right\}$, that is $|\Psi(t)\rangle \equiv\left|\Psi\left(\lambda_{1}, \lambda_{2}, \ldots\right)\right\rangle$. The the Lagrangian can be rewritten as 

$$
L=i \sum_{k}\langle\Psi| \frac{\partial|\Psi\rangle}{\partial \lambda_{k}} \dot{\lambda}_{k}-\langle\Psi|H| \Psi\rangle
$$

which is a function of parameters $\{\lambda_{k}\}$ and their time derivatives $\left\{\dot{\lambda}_{k}=\left[d \lambda_{k} / d t\right]\right\}$. After applying the [Euler-Lagrange equation](https://en.wikipedia.org/wiki/Euler–Lagrange_equation#Statement), the evolution of parameters is

$$
\sum_{q} A_{k, q} \dot{\lambda}_{q}=V_{k}
$$

where

$$
A_{k, q}=i  \frac{\partial\langle\Psi|}{\partial \lambda_{k}} \frac{\partial|\Psi\rangle}{\partial \lambda_{q}}+\text { H.c. } \quad \text{and} \quad V_{k}= \frac{\partial\langle\Psi|}{\partial \lambda_{k}} H|\Psi\rangle+\text { H.c. }
$$

## 3. Variational algorithm on a hybrid computer

The variational method avoids the exponential space problem by considering trial states from a physically motivated small
subset of the exponentially large Hilbert space. Here we consider trial states that can be directly prepared in the quantum computer; i.e., states can be expressed as $|\Psi\rangle=R|\overline{0}\rangle$, where $|\overline{0}\rangle$ is an initial state of the quantum computer and $R$ is a sequence of $N$ quantum gates determined by parameters $\left\{\lambda_{k}\right\}$, that is

$$
R=R_{N}\left(\lambda_{N}\right) \ldots R_{k}\left(\lambda_{k}\right) \ldots R_{1}\left(\lambda_{1}\right).
$$

<!-- If N is smaller than the dimension of the Hilbert space, trial states $|\Psi\rangle$ only span a submanifold of the Hilbert space. Therefore, Eq.\eqref{main} approximates -->
In conventional quantum computing, each unitary gate involves a small subset of qubits, typically one or two. Thefore, the derivative of the unitary can be efficiently decomposed via 

$$
\frac{d R_{k}}{d \lambda_{k}}=\sum_{i} f_{k, i} R_{k} \sigma_{k, i},
$$

where $\sigma_{k, i}$ are also unitary operators and $f_{k, i}$ are complex coefficients. For most single- and two-qubit gates $R_{k}$, we find that there are only one or two terms in this expression, and $\sigma_{k, i}$ is also a one-qubit or two-qubit gate. 

<details>
    <summary>Click to Expand: Example</summary>
For the case when 
$$
R_{k}(\lambda_{k})=e^{-i \lambda_{k} \sigma / 2},$$ 
with a one- or two-qubit Hermitian operator $\sigma$, we have
$$
\frac{\partial R_{k}(\lambda_{k})}{\partial \lambda_{k}}=-\frac{i}{2} \cdot \sigma \cdot e^{-i \lambda_{k} \sigma / 2},
$$
where $f_{k}=-i / 2$ and $\sigma_{k}=\sigma$.
</details>


With the definition of the trial state we can rewrite the derivative of the state as

$$
\frac{\partial|\Psi\rangle}{\partial \lambda_{k}}=\sum_{i} f_{k, i} R_{k, i}|\overline{0}\rangle,
$$

where
$$
R_{k, i}=R_{N} R_{N-1} \cdots R_{k+1} R_{k} \sigma_{k, i} \cdots R_{2} R_{1}.
$$

Then, based on the definition of the Hamiltonian, the elements of $A$ and $V$ can be expressed as

$$
A_{k, q}=\sum_{i, j}\left(f_{k, i}^{*} f_{q, j}\langle\overline{0}|R_{k, i}^{\dagger} R_{q, j}| \overline{0}\rangle\right) \quad \text{and} \quad V_{k}=\sum_{i, j}\left(f_{k, i}^{*} h_{j}\langle\overline{0}|R_{k, i}^{\dagger} \sigma_{j} R| \overline{0}\rangle\right).
$$

Note that each term is in the form $a \operatorname{Re}(e^{i \theta}\langle\overline{0}|U| \overline{0}\rangle)$ where the amplitude $a$ and phase $\theta$ are determined by either $i f_{k, i}^{*} f_{q, j}$ or $f_{k, i}^{*} h_{j}$, and $U$ is a unitary operator equal to either $R_{k, i}^{\dagger} R_{q, j}$ or $R_{k, i}^{\dagger} \sigma_{j} R$. Such a term can be evaluated using the following quantum circuit

<img src="img/circuit.png" alt="Drawing" style="width: 700px;"/>
<font size="2"><center> Figure extracted from the paper "Theory of variational quantum simulation" </center> <font size="5"></font>

This circuit needs an ancillary qubit initialized in the state $\left(|0\rangle+e^{i \theta}|1\rangle\right) / \sqrt{2}$ and a register initialized in the state $|\overline{0}\rangle$. The ancillary qubit is measured in the $\{|+\rangle,|-\rangle\}$ basis after a sequence of quantum gates on the register and two controlled gates, in which the ancillary qubit is the control qubit. The value of each term is given by $\operatorname{Re}(e^{i \theta}\langle\overline{0}|U| \overline{0}\rangle)= 2P_{+} - 1$, where $P_{+}$ is the probability that the qubit is in the state $| + \rangle$.

## 4. Implementation using Qiskit

## 5. Simulations for quantum Ising

## 6.  Experiments for quantum Ising model

## 7. Discussion and outlook

There are some improvements that could be made on the package. 
- We are currently implementing the variational forms using the Qiskit function <i>HamiltonianGate </i> applied to a string of Paulis $\sigma_k$ and a parameter $\lambda_k$. This function constructs $e^{-i \lambda_k \sigma_k}$ and decomposes it into basic gates. This is not necessarly efficient when the number of qubits is high, so we could improve this constructing variational forms that are efficient for specific each problem. 
- The same algorithm can be used to perform imaginary time evolution. This is useful to find ground states of Hamiltonians.
- We could test our package with different Hamiltonians and with a higher number of qubits.
- We could modify the parameters in the Hamiltonians to find phase transitions in the systems.

## References

[1] Ying Li and Simon C. Benjamin. Efficient variational quantum simulator incorporating active error minimization. $\textit{Phys. Rev. X}$, 7:021050, Jun 2017. DOI: https://doi.org/10.1103/PhysRevX.7.021050. URL: https://link.aps.org/doi/10.1103/PhysRevX.7.021050.

[2] Yuan, X., Endo, S., Zhao, Q., Li, Y. & Benjamin, S. C. Theory of variational quantum simulation. $\textit{Quantum 3}$, 191 (2019). DOI: https://doi.org/10.22331/q-2019-10-07-191. URL:https://quantum-journal.org/papers/q-2019-10-07-191/
